## Evaluation (Lebedev dataset), tests on WMD similarity

In [1]:
from gensim.models import KeyedVectors
import numpy as np
import os
from tqdm.auto import tqdm
import pandas as pd

import functions as f 

In [2]:
model = KeyedVectors.load('/Users/nikolaevaanna/Downloads/187/model.model')
# model.init_sims(replace=True)

# Лебедев texts

## 1.1 Bipartite + Subgraphs

In [65]:
tau_candidates = [round(i,2) for i in list(0.01 * np.arange(100, 270, 20))]
tau_candidates

[1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6]

In [66]:
tau_dict_pk = {i:[] for i in tau_candidates}
tau_dict_windowdiff = {i:[] for i in tau_candidates}

In [3]:
text_paths = sorted(["texts/lebedev/" + i for i in os.listdir("texts/lebedev")])

In [4]:
text_paths

['texts/lebedev/01.txt',
 'texts/lebedev/02.txt',
 'texts/lebedev/03.txt',
 'texts/lebedev/04.txt',
 'texts/lebedev/05.txt',
 'texts/lebedev/06.txt',
 'texts/lebedev/07.txt']

Finding optimal tau

In [69]:
for path in tqdm(text_paths[:-3]):
        
    for tau in tau_candidates:
        
        (sents, 
         real_paragraphs, 
         pred_segment_indices, 
         pred_paragraphs) = f.segmentize_bipartite_subgraphs(path, 
                                                    model=model,
                                                    tau=tau,
                                                    wmd=False)
        
        print(len(real_paragraphs), len(pred_segment_indices))
        if len(pred_segment_indices) < len(real_paragraphs)/2 or len(pred_segment_indices) > len(real_paragraphs)*2:
            continue
        
        k = round(0.5*len(sents)/len(real_paragraphs))
        scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)
            
        
        tau_dict_pk[tau].append(scores["pk"])
        tau_dict_windowdiff[tau].append(scores["windowdiff"])

Lemmatizing sentences
10 7
Lemmatizing sentences
10 8
Lemmatizing sentences
10 12
Lemmatizing sentences
10 15
Lemmatizing sentences
10 18
Lemmatizing sentences
10 18
Lemmatizing sentences
10 25
Lemmatizing sentences
10 26
Lemmatizing sentences
10 28
Lemmatizing sentences
7 5
Lemmatizing sentences
7 6
Lemmatizing sentences
7 7
Lemmatizing sentences
7 9
Lemmatizing sentences
7 10
Lemmatizing sentences
7 13
Lemmatizing sentences
7 15
Lemmatizing sentences
7 16
Lemmatizing sentences
7 16
Lemmatizing sentences
7 2
Lemmatizing sentences
7 3
Lemmatizing sentences
7 3
Lemmatizing sentences
7 5
Lemmatizing sentences
7 7
Lemmatizing sentences
7 9
Lemmatizing sentences
7 12
Lemmatizing sentences
7 17
Lemmatizing sentences
7 19
Lemmatizing sentences
7 4
Lemmatizing sentences
7 4
Lemmatizing sentences
7 4
Lemmatizing sentences
7 5
Lemmatizing sentences
7 6
Lemmatizing sentences
7 9
Lemmatizing sentences
7 12
Lemmatizing sentences
7 13
Lemmatizing sentences
7 15


In [71]:
tau_dict_pk_short = {k:v for k,v in tau_dict_pk.items() if len(v) == 4}
tau_dict_pk_short

{1.6: [0.32038834951456313,
  0.37254901960784315,
  0.42592592592592593,
  0.6086956521739131],
 1.8: [0.39805825242718446,
  0.35294117647058826,
  0.5370370370370371,
  0.5434782608695652],
 2.0: [0.4174757281553398,
  0.29411764705882354,
  0.46296296296296297,
  0.43478260869565216]}

In [72]:
pd.DataFrame(tau_dict_pk_short)

,1.6,1.8,2.0
0,0.320388,0.398058,0.417476
1,0.372549,0.352941,0.294118
2,0.425926,0.537037,0.462963
3,0.608696,0.543478,0.434783


In [73]:
tau_dict_pk_mean = {k:np.mean(v) for k,v in tau_dict_pk_short.items()}
tau_dict_pk_mean

{1.6: 0.4318897368055613, 1.8: 0.45787868170109375, 2.0: 0.4023347367181946}

In [ ]:
# for pk the best tau is 2

In [74]:
tau_dict_windowdiff_short = {k:v for k,v in tau_dict_windowdiff.items() if len(v) == 4}
pd.DataFrame(tau_dict_windowdiff_short)

,1.6,1.8,2.0
0,0.475728,0.563107,0.582524
1,0.431373,0.411765,0.431373
2,0.462963,0.574074,0.555556
3,0.652174,0.586957,0.543478


In [75]:
{k:np.mean(v) for k,v in tau_dict_windowdiff_short.items()}

{1.6: 0.5055593950914637, 1.8: 0.5339755244530155, 2.0: 0.5282326593223472}

In [93]:
# for windowdiff the best tau is 1.6
# let tau be 1.8 

## Testing (3 texts)

In [113]:
tau = 1.8

In [114]:
all_scores = {"pk":[], "windowdiff":[]}
real_pars = []
pred_pars = []


for path in tqdm(text_paths[-3:]):
        
    (sents, 
         real_paragraphs, 
         pred_segment_indices, 
         pred_paragraphs) = f.segmentize_bipartite_subgraphs(path, 
                                                    model=model,
                                                    tau=tau,
                                                    wmd=False)
        
    print(len(real_paragraphs), len(pred_segment_indices))
        
    k = round(0.5*len(sents)/len(real_paragraphs))
    scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)     
    
        
    all_scores["pk"].append(scores["pk"])
    all_scores["windowdiff"].append(scores["windowdiff"])
    
    real_pars.append(real_paragraphs)
    pred_pars.append(pred_paragraphs)

Lemmatizing sentences
12 12
Lemmatizing sentences
9 15
Lemmatizing sentences
8 5


In [115]:
all_scores

{'pk': [0.5108695652173914, 0.6263736263736264, 0.35185185185185186],
 'windowdiff': [0.5217391304347826, 0.7032967032967034, 0.37037037037037035]}

In [116]:
np.mean(all_scores["pk"])

0.49636501448095655

In [117]:
np.mean(all_scores["windowdiff"])

0.5318020680339521

In [118]:
pred_pars[1]

[['В Австрии есть прекрасный старинный замок на скале - Хохостервиц, про который существует не менее древняя легенда.',
  'Войска под предводительством Маргариты Маульташ (типа самой уродливой женщины всех времен и народов) осадили замок.'],
 ['Решили брать измором, но стояли так долго, что начали голодать сами.',
  'В последний момент комендант замка решил нафаршировать остатками зерна последнюю корову и катапультировать ее на врага.',
  'Это была последняя еда оборонявшихся.',
  'Голодные нападавшие решили, что раз у народа в осаде так много жратвы, то дальше стоять нет смысла и съебались.',
  'Кажется, что это управленческий парадокс - выкинуть последнюю еду, когда пиздец нечего есть.',
  'Но мораль простая: когда нечего терять - блефуй.',
  'Очень интересно в языке устроено разделение на важное и ничтожное.',
  'Важное - большое или высокое.',
  'Ничтожное - маленькое или низкое.',
  'Вершина власти.'],
 ['Унижаться.', 'С высоты положения.'],
 ['Низко упасть.', 'Возвеличивать прави

## Testing (all texts)

In [6]:
all_scores = {"pk":[], "windowdiff":[]}
real_pars = []
pred_pars = []
tau = 1.8

for path in tqdm(text_paths):
        
    (sents, 
         real_paragraphs, 
         pred_segment_indices, 
         pred_paragraphs) = f.segmentize_bipartite_subgraphs(path, 
                                                    model=model,
                                                    tau=tau,
                                                    wmd=False)
        
    print(len(real_paragraphs), len(pred_segment_indices))
        
    k = round(0.5*len(sents)/len(real_paragraphs))
    scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)     
    
        
    all_scores["pk"].append(scores["pk"])
    all_scores["windowdiff"].append(scores["windowdiff"])
    
    real_pars.append(real_paragraphs)
    pred_pars.append(pred_paragraphs)

Lemmatizing sentences
10 18
Lemmatizing sentences
7 10
Lemmatizing sentences
7 7
Lemmatizing sentences
7 6
Lemmatizing sentences
12 12
Lemmatizing sentences
9 15
Lemmatizing sentences
8 5


In [7]:
all_scores

{'pk': [0.39805825242718446,
  0.35294117647058826,
  0.5370370370370371,
  0.5434782608695652,
  0.5108695652173914,
  0.6263736263736264,
  0.35185185185185186],
 'windowdiff': [0.5631067961165048,
  0.4117647058823529,
  0.5740740740740741,
  0.5869565217391305,
  0.5217391304347826,
  0.7032967032967034,
  0.37037037037037035]}

In [8]:
np.mean(all_scores["pk"])

0.47437282432103495

In [9]:
np.mean(all_scores["windowdiff"])

0.5330440431305598

# 1.2 Clustering

## Test

In [120]:
# we don't have hyperparameters, so just cluster words in texts
all_scores = {"pk":[], "windowdiff":[]}
real_pars = []
pred_pars = []

In [121]:
for path in tqdm(text_paths[-3:]):
        
    (sents, 
         real_paragraphs, 
         pred_segment_indices, 
         pred_paragraphs) = f.segmentize_by_clustering(path, 
                                                    model=model)
        
    print(len(real_paragraphs), len(pred_segment_indices))
        
    k = round(0.5*len(sents)/len(real_paragraphs))
    scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)     
    
        
    all_scores["pk"].append(scores["pk"])
    all_scores["windowdiff"].append(scores["windowdiff"])
    
    real_pars.append(real_paragraphs)
    pred_pars.append(pred_paragraphs)

Lemmatizing sentences
12 15
Lemmatizing sentences
9 13
Lemmatizing sentences
8 10


In [122]:
all_scores

{'pk': [0.41304347826086957, 0.5274725274725275, 0.2777777777777778],
 'windowdiff': [0.42391304347826086, 0.5494505494505495, 0.2962962962962963]}

In [123]:
np.mean(all_scores["pk"])

0.4060979278370582

In [124]:
np.mean(all_scores["windowdiff"])

0.42321996307503557

In [125]:
pred_pars[0]

[['Пиздец, больше всего на свете меня бесит, что в кино продают треульные начос и круглые банки с сыром.',
  'И начос в эту банку залезает только одним уголочком.',
  'И ты должен каждый раз откусывать уголок, чтобы начос приобрел более прямоугольную форму и начал в банку с сыром залезать.',
  'Вот почему не сделать банки с сыром такой формы, чтобы начос в них сразу было удобно макать?',
  'Пиздец, самая большая тупость на свете - это концепция "здоровой еды".',
  'Епта, если не пить серную кислоту и не заедать ее асбестом, любая еда - здоровая.',
  'Когда человек поел, он живет.'],
 ['Ну а если не будет есть, то в какой-то момент помрет.У каждого поколения есть свои пищевые стереотипы - то надо есть жир, то не надо, то слова надо.',
  'Единственное, что точно известно: надо срать на все мнения и есть любую еду.',
  'Чем разнообразнее, тем лучше.',
  'Нет какого-то одного невероятно полезного продукта.',
  'Ягоды годжи не спасут никого, а если их есть каждый день, то еще и заебут.',
  

## Test (all data)

In [11]:
all_scores = {"pk":[], "windowdiff":[]}
real_pars = []
pred_pars = []

In [12]:
for path in tqdm(text_paths):
        
    (sents, 
         real_paragraphs, 
         pred_segment_indices, 
         pred_paragraphs) = f.segmentize_by_clustering(path, 
                                                    model=model)
        
    print(len(real_paragraphs), len(pred_segment_indices))
        
    k = round(0.5*len(sents)/len(real_paragraphs))
    scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)     
    
        
    all_scores["pk"].append(scores["pk"])
    all_scores["windowdiff"].append(scores["windowdiff"])
    
    real_pars.append(real_paragraphs)
    pred_pars.append(pred_paragraphs)

Lemmatizing sentences
10 17
Lemmatizing sentences
7 11
Lemmatizing sentences
7 11
Lemmatizing sentences
7 10
Lemmatizing sentences
12 16
Lemmatizing sentences
9 13
Lemmatizing sentences
8 10


In [17]:
pred_pars

[[['Дорожные знаки распространяются на пространство слева от них (если смотреть на знаки).',
   'Это довольно простое правило, которое не изложено понятным образом в ПДД, его не проходят в автошколах, не запоминают ученики.',
   'Скажем, мы едем по бульвару, в центре которого стоит кирпич.',
   'Почему нельзя ехать по левой стороне дороги?',
   'Потому что на левую часть дороги распространяется действие знака.',
   'Если бы знак стоял с самого права, то нельзя было бы ехать по правой дороге, а по левой как раз можно было бы.',
   'Потому что знак действует на всю ширину дороги, но не распространяется на то, что будет после обочины.'],
  ['Простое знание этого правила невероятно помогает во время езды.',
   'Потому что в реальной жизни не нужно решать шахматные задачки из билетов, а нужно просто ехать не сильно думая, но опираться на нерушимые и простые правила.',
   'В кабине пилота они различаются очень сильно - в Боингах стоят штурвалы, их натягивают на себя, а в Аэробусах стоят джой

In [13]:
all_scores

{'pk': [0.5145631067961165,
  0.5294117647058824,
  0.5370370370370371,
  0.391304347826087,
  0.4891304347826087,
  0.5274725274725275,
  0.2777777777777778],
 'windowdiff': [0.5436893203883495,
  0.5686274509803921,
  0.5370370370370371,
  0.4782608695652174,
  0.5,
  0.5494505494505495,
  0.2962962962962963]}

In [14]:
np.mean(all_scores["pk"])

0.46667099948543384

In [15]:
np.mean(all_scores["windowdiff"])

0.4961945033882631

# WMD (Lebedev)

In [146]:
tau_candidates = [round(i,2) for i in list(0.01 * np.arange(90, 200, 3))]
tau_candidates

[0.9,
 0.93,
 0.96,
 0.99,
 1.02,
 1.05,
 1.08,
 1.11,
 1.14,
 1.17,
 1.2,
 1.23,
 1.26,
 1.29,
 1.32,
 1.35,
 1.38,
 1.41,
 1.44,
 1.47,
 1.5,
 1.53,
 1.56,
 1.59,
 1.62,
 1.65,
 1.68,
 1.71,
 1.74,
 1.77,
 1.8,
 1.83,
 1.86,
 1.89,
 1.92,
 1.95,
 1.98]

In [147]:
tau_dict_pk = {i:[] for i in tau_candidates}
tau_dict_windowdiff = {i:[] for i in tau_candidates}

In [148]:
for path in tqdm(text_paths[:-3]):
        
    for tau in tau_candidates:
        
        (dists, sents, 
         real_paragraphs, 
         pred_segment_indices, 
         pred_paragraphs) = f.segmentize_bipartite_subgraphs(path, 
                                                    model=model,
                                                    tau=tau,
                                                    wmd=True)
        
        print(len(real_paragraphs), len(pred_segment_indices))
        if len(pred_segment_indices) < len(real_paragraphs)/2 or len(pred_segment_indices) > len(real_paragraphs)*2:
            continue
        
        k = round(0.5*len(sents)/len(real_paragraphs))
        scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)
            
        
        tau_dict_pk[tau].append(scores["pk"])
        tau_dict_windowdiff[tau].append(scores["windowdiff"])

Lemmatizing sentences
10 20
Lemmatizing sentences
10 22
Lemmatizing sentences
10 22
Lemmatizing sentences
10 29
Lemmatizing sentences
10 30
Lemmatizing sentences
10 30
Lemmatizing sentences
10 31
Lemmatizing sentences
10 28
Lemmatizing sentences
10 30
Lemmatizing sentences
10 38
Lemmatizing sentences
10 33
Lemmatizing sentences
10 28
Lemmatizing sentences
10 23
Lemmatizing sentences
10 18
Lemmatizing sentences
10 15
Lemmatizing sentences
10 13
Lemmatizing sentences
10 10
Lemmatizing sentences
10 4
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing sentences
10 1
Lemmatizing

In [149]:
tau_dict_pk_short = {k:v for k,v in tau_dict_pk.items() if len(v) == 4}
tau_dict_pk_short

{0.9: [0.5825242718446602,
  0.43137254901960786,
  0.42592592592592593,
  0.6304347826086957],
 1.29: [0.3883495145631068,
  0.43137254901960786,
  0.42592592592592593,
  0.6086956521739131],
 1.32: [0.32038834951456313,
  0.39215686274509803,
  0.42592592592592593,
  0.6086956521739131]}

In [150]:
{k:np.mean(v) for k,v in tau_dict_pk_short.items()}

{0.9: 0.5175643823497225, 1.29: 0.46358591042063846, 1.32: 0.436791697589875}

In [151]:
tau_dict_windowdiff_short = {k:v for k,v in tau_dict_windowdiff.items() if len(v) == 4}
tau_dict_windowdiff_short

{0.9: [0.6893203883495146, 0.49019607843137253, 0.5, 0.6739130434782609],
 1.29: [0.5436893203883495,
  0.5490196078431373,
  0.46296296296296297,
  0.7608695652173914],
 1.32: [0.46601941747572817,
  0.47058823529411764,
  0.46296296296296297,
  0.6956521739130435]}

In [170]:
{k:np.mean(v) for k,v in tau_dict_windowdiff_short.items()}

{0.9: 0.588357377564787, 1.29: 0.5791353641029603, 1.32: 0.5238056974114631}

In [ ]:
# tau = 1.32

# Testing

In [179]:
all_scores = {"pk":[], "windowdiff":[]}
real_pars = []
pred_pars = []
tau=1.32

for path in tqdm(text_paths[-3:]):
        
    print(path)
    (wmd_distances, sents, real_paragraphs, pred_segment_indices, pred_paragraphs) = f.segmentize_bipartite_subgraphs(path, 
                                                                                                       model=model,
                                                                                                        tau=tau,
                                                                                                        wmd=True)
        
    print(len(real_paragraphs), len(pred_segment_indices))
        
    k = round(0.5*len(sents)/len(real_paragraphs))
    scores = f.evaluate(f.gold_segment_indices(real_paragraphs), pred_segment_indices, k=k)     
    
        
    all_scores["pk"].append(scores["pk"])
    all_scores["windowdiff"].append(scores["windowdiff"])
    
    real_pars.append(real_paragraphs)
    pred_pars.append(pred_paragraphs)

texts/lebedev/05.txt
Lemmatizing sentences
12 10
texts/lebedev/06.txt
Lemmatizing sentences
9 8
texts/lebedev/07.txt
Lemmatizing sentences
8 6


In [180]:
all_scores

{'pk': [0.5543478260869565, 0.6043956043956044, 0.35185185185185186],
 'windowdiff': [0.5978260869565217, 0.6153846153846154, 0.4074074074074074]}

In [181]:
np.mean(all_scores["pk"])

0.5035317607781377

In [182]:
np.mean(all_scores["windowdiff"])

0.5402060365828482